# Asymmetric Grs-Pyr Binary Garnet soln (van-Laar)
- based on Holland2018

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
from scipy import spatial
from scipy import interpolate
import matplotlib.pyplot as plt

import vanlaar

## Create Pyrope-Grossular Binary Soln models
- uses parameter values from Holland2018

In [ ]:
R = 8.3145

# Pressure appears to be in kbars???

Wh = 45.4*1e3
Ws = .01*1e3
Wv = .04*1e3

alpha = 2.5
mult = 3


In [ ]:
regular = vanlaar.RegularSoln(Wh, Wv=Wv, Ws=Ws, mult=mult)
asymm = vanlaar.AsymmSoln(Wh, Wv=Wv, Ws=Ws, alpha=alpha, mult=mult)

In [ ]:
P=0
T0_crit = 750

T = T0_crit*.9

## Find solvus using common tangent

In [ ]:
regular.test_solvus_finder(T=T, P=P)

In [ ]:
asymm.test_solvus_finder(T=T, P=P)

# Solvus Calculation

In [ ]:
def plot_solvus(T, X_solvus, label='', fmt='k--', figure=None):

    if figure is None:
        figure = plt.figure()
        
    plt.plot(X_solvus[:,0], T, fmt, label=label)
    plt.plot(X_solvus[:,1], T, fmt)

    plt.xlim(-.01,1.01)
    plt.xlabel('X')
    plt.ylabel('T [K]')
    
    if label!='':
        plt.legend()
        
    return figure

In [ ]:
# T = np.linspace(.5*T0_crit,3*T0_crit,301)
T = np.linspace(400,870, 3601)
P0=1e-3

N=6001
upsamp_fac = 1
allow_interp = False
Phi=30

In [ ]:
# X_solvus = regular.solvus(T, P=P0)
X_solvus_asymm = asymm.solvus(T, P=P0, N=N, allow_interp=allow_interp)

# X_solvus_hi = regular.solvus(T, P=Phi)
X_solvus_asymm_hi = asymm.solvus(T, P=Phi, N=N, allow_interp=allow_interp)

In [ ]:
def reformat_solvus(X_solvus, T):
    Xsolv = np.hstack((X_solvus[:,0],X_solvus[::-1,1]))
    Tsolv = np.hstack((T, T[::-1]))
    # ind_solv = ~(np.isnan(Tsolv) or np.isnan(Xsolv))
    ind_solv = ~(np.isnan(Xsolv)|np.isnan(Tsolv))

    Xsolv = Xsolv[ind_solv]
    Tsolv = Tsolv[ind_solv]
    
    return Xsolv, Tsolv



In [ ]:
Xsolv, Tsolv = reformat_solvus(X_solvus_asymm,  T)
Xsolv_hi, Tsolv_hi = reformat_solvus(X_solvus_asymm_hi,  T)

In [ ]:
df_solvus = pd.DataFrame({'X':Xsolv, 'T':Tsolv})
df_solvus_3 = pd.DataFrame({'X':Xsolv_hi, 'T':Tsolv_hi})
# df_solvus.to_csv('data/solvus_Holland2018.csv')
# df_solvus_3.to_csv('data/solvus_3_Holland2018.csv')

In [ ]:
plt.figure()
plt.plot(Xsolv, Tsolv,  'k-')
plt.plot(Xsolv_hi, Tsolv_hi,  'r-')

## Explore accurately pinpointing solvus edges w/o convex hull

In [ ]:
T=760
X = np.linspace(0,1,1001)
dX = X[1]-X[0]

In [ ]:
G_mix = asymm.gibbs_mix(X, T, P=.001)

dGdX = np.gradient(G_mix, dX)

mu1 = G_mix - dGdX*X
mu2 = G_mix + dGdX*(1-X)

plt.figure()

# plt.plot(X, mu1, 'k-')
plt.plot(X, mu2, 'r-')

plt.ylim(-1300,-1000)

plt.figure()

plt.plot(X, mu1, 'k-')
# plt.plot(X, mu2, 'r-')

# plt.ylim(-9000,-6000)


plt.figure()
plt.plot(X, G_mix, '-')



In [ ]:
dmu1 = mu1[:,np.newaxis]-mu1[np.newaxis,:]
dmu2 = mu2[:,np.newaxis]-mu2[np.newaxis,:]

plt.figure()
plt.contour(X, X, np.abs(dmu1), 1000)

plt.colorbar()
plt.figure()
plt.contour(X, X, dmu2, 1000)